
# 🌼 대규모 LLM을 활용한 지식 챗봇 개발 - 1차시(24.11.21)

---


In [1]:
import os
import shutil
import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
lines = pd.read_csv('./data/fra.txt', names=['src', 'tar', 'lic'], sep='\t')
lines.head()

,src,tar,lic
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [3]:
del lines['lic']

In [4]:
print(f'전체 데이터 개수: {len(lines)}')

전체 데이터 개수: 232736


In [5]:
lines.head()

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


In [6]:
lines = lines[:60000]
# 데이터 개수 줄이기
lines.sample(10)

,src,tar
38511,You aren't invited.,Vous n'êtes pas invité.
18594,We're very busy.,Nous sommes fort occupés.
59776,Nothing is impossible.,Rien n'est impossible.
32858,He arrived in time.,Il est arrivé à temps.
55895,Don't touch the stove.,Ne touchez pas le poêle.
5360,I might stay.,Il se pourrait que je reste.
15992,I was very busy.,J'étais bien occupé.
25282,Am I a bad person?,Suis-je quelqu'un de mauvais ?
9121,Show yourself.,Montre-toi !
44301,This guy is a loser.,Ce mec est un nullard.


In [7]:
lines.tar = lines.tar.apply(lambda x: '\t '+ x + ' \n')
# 시작을 의미하는 sos , 끝을 의미하는 eos

In [8]:
lines.tar

0                              \t Va ! \n
1                           \t Marche. \n
2                        \t En route ! \n
3                           \t Bouge ! \n
4                           \t Salut ! \n
                       ...               
59995      \t Elle aime l'enseignante. \n
59996          \t Elle aime le maitre. \n
59997       \t Elle aime la maitresse. \n
59998    \t Elle vécut une longue vie. \n
59999          \t Elle vit à New York. \n
Name: tar, Length: 60000, dtype: object

In [9]:
lines.sample(10)

,src,tar
44654,Tom is on his phone.,\t Tom est au téléphone. \n
22381,No one disagreed.,\t Personne ne fut en désaccord. \n
33614,I failed miserably.,\t J'ai lamentablement échoué. \n
15788,I outwitted him.,\t J'ai été plus maline que lui. \n
6678,Try to sleep.,\t Essayez de dormir. \n
22585,"Red wine, please.","\t Du vin rouge, s'il vous plait. \n"
22961,The apple is red.,\t La pomme est rouge. \n
46016,Would you chill out?,\t Voudrais-tu te détendre ? \n
47287,Do you understand me?,\t Me comprends-tu ? \n
53428,We have lots of time.,\t Nous avons beaucoup de temps. \n


In [10]:
src_vocab = set()
for line in lines['src']:
    for char in line:
        src_vocab.add(char)
        # src 문자 집합 구축

In [11]:
print(src_vocab)

{'M', 'q', "'", 'l', 'P', '0', '7', '"', 'S', 'B', 'b', 'X', 'e', 'I', 'i', '-', 'é', 'u', '3', 'N', 'G', 'C', '5', 'p', '8', 'F', '€', ' ', '$', 'z', 'O', 'w', 's', 'n', 'A', 'o', 'R', 'y', '6', ',', 't', 'r', '%', '4', 'x', 'H', 'g', 'k', 'L', 'f', 'j', '’', 'W', 'h', 'Y', 'Q', '/', 'ï', '9', 'v', 'm', 'T', '&', 'a', 'D', 'E', 'c', 'V', 'U', '!', 'J', 'd', ':', '1', 'K', 'Z', '?', '2', '.'}


In [12]:
target_vocab = set()
for line in lines['tar']:
    for char in line:
        target_vocab.add(char)
        # tar 문자 집합 구축

In [13]:
print(target_vocab)

{'M', '«', '‽', 'q', "'", 'l', 'P', '0', '7', '"', 'S', 'B', 'ê', 'b', 'X', 'e', 'I', 'i', 'î', 'œ', '-', 'é', 'u', '3', 'N', 'G', 'C', '5', 'p', '8', 'ë', 'F', ' ', '$', 'û', 'z', 'O', 'w', '\n', 'Ô', 's', 'n', 'A', 'o', 'R', '\xa0', 'y', 'Ê', '6', 'ç', ',', 't', 'r', '%', '4', 'x', 'H', 'g', '\u202f', 'k', 'L', 'è', 'j', 'f', '’', 'W', 'h', '\t', '\u2009', 'ï', 'Q', 'Y', '9', 'v', 'm', 'T', '&', 'a', 'D', 'É', 'Ç', 'E', 'ô', 'c', 'V', 'U', 'à', '!', 'J', 'ù', 'd', 'À', ':', '1', 'K', '?', '2', 'â', '‘', '.', '»'}


In [14]:
src_vocab_size = len(src_vocab) + 1
print(f'src 문장의 문자 집합 크기: {src_vocab_size}')
# 인코더에 데이터를 넣을 때 사용될 차원의 크기

src 문장의 문자 집합 크기: 80


In [15]:
tar_vocab_size = len(target_vocab) + 1
print(f'tar 문장의 문자 집합 크기: {tar_vocab_size}')
# 디코더에 데이터를 넣을 때 사용될 차원의 크기

tar 문장의 문자 집합 크기: 102


In [16]:
print(src_vocab)

{'M', 'q', "'", 'l', 'P', '0', '7', '"', 'S', 'B', 'b', 'X', 'e', 'I', 'i', '-', 'é', 'u', '3', 'N', 'G', 'C', '5', 'p', '8', 'F', '€', ' ', '$', 'z', 'O', 'w', 's', 'n', 'A', 'o', 'R', 'y', '6', ',', 't', 'r', '%', '4', 'x', 'H', 'g', 'k', 'L', 'f', 'j', '’', 'W', 'h', 'Y', 'Q', '/', 'ï', '9', 'v', 'm', 'T', '&', 'a', 'D', 'E', 'c', 'V', 'U', '!', 'J', 'd', ':', '1', 'K', 'Z', '?', '2', '.'}


In [17]:
src_to_index = {}

for i, word in enumerate(src_vocab):
    src_to_index[word] = i + 1  # padding
    # 정수 인덱싱을 위한 인코딩 작업

In [18]:
print(src_to_index)

{'M': 1, 'q': 2, "'": 3, 'l': 4, 'P': 5, '0': 6, '7': 7, '"': 8, 'S': 9, 'B': 10, 'b': 11, 'X': 12, 'e': 13, 'I': 14, 'i': 15, '-': 16, 'é': 17, 'u': 18, '3': 19, 'N': 20, 'G': 21, 'C': 22, '5': 23, 'p': 24, '8': 25, 'F': 26, '€': 27, ' ': 28, '$': 29, 'z': 30, 'O': 31, 'w': 32, 's': 33, 'n': 34, 'A': 35, 'o': 36, 'R': 37, 'y': 38, '6': 39, ',': 40, 't': 41, 'r': 42, '%': 43, '4': 44, 'x': 45, 'H': 46, 'g': 47, 'k': 48, 'L': 49, 'f': 50, 'j': 51, '’': 52, 'W': 53, 'h': 54, 'Y': 55, 'Q': 56, '/': 57, 'ï': 58, '9': 59, 'v': 60, 'm': 61, 'T': 62, '&': 63, 'a': 64, 'D': 65, 'E': 66, 'c': 67, 'V': 68, 'U': 69, '!': 70, 'J': 71, 'd': 72, ':': 73, '1': 74, 'K': 75, 'Z': 76, '?': 77, '2': 78, '.': 79}


In [19]:
tar_to_index = {}

for i, word in enumerate(target_vocab):
    tar_to_index[word] = i+1

In [20]:
print(tar_to_index)

{'M': 1, '«': 2, '‽': 3, 'q': 4, "'": 5, 'l': 6, 'P': 7, '0': 8, '7': 9, '"': 10, 'S': 11, 'B': 12, 'ê': 13, 'b': 14, 'X': 15, 'e': 16, 'I': 17, 'i': 18, 'î': 19, 'œ': 20, '-': 21, 'é': 22, 'u': 23, '3': 24, 'N': 25, 'G': 26, 'C': 27, '5': 28, 'p': 29, '8': 30, 'ë': 31, 'F': 32, ' ': 33, '$': 34, 'û': 35, 'z': 36, 'O': 37, 'w': 38, '\n': 39, 'Ô': 40, 's': 41, 'n': 42, 'A': 43, 'o': 44, 'R': 45, '\xa0': 46, 'y': 47, 'Ê': 48, '6': 49, 'ç': 50, ',': 51, 't': 52, 'r': 53, '%': 54, '4': 55, 'x': 56, 'H': 57, 'g': 58, '\u202f': 59, 'k': 60, 'L': 61, 'è': 62, 'j': 63, 'f': 64, '’': 65, 'W': 66, 'h': 67, '\t': 68, '\u2009': 69, 'ï': 70, 'Q': 71, 'Y': 72, '9': 73, 'v': 74, 'm': 75, 'T': 76, '&': 77, 'a': 78, 'D': 79, 'É': 80, 'Ç': 81, 'E': 82, 'ô': 83, 'c': 84, 'V': 85, 'U': 86, 'à': 87, '!': 88, 'J': 89, 'ù': 90, 'd': 91, 'À': 92, ':': 93, '1': 94, 'K': 95, '?': 96, '2': 97, 'â': 98, '‘': 99, '.': 100, '»': 101}


In [21]:
encoder_input = []
for line in lines['src']:
    encoded_line = []
    for char in line:
        encoded_line.append(src_to_index[char])
    encoder_input.append(encoded_line)

print(f'src 문장의 정수 인코딩 {encoder_input[:5]}')

src 문장의 정수 인코딩 [[21, 36, 79], [21, 36, 79], [21, 36, 79], [21, 36, 79], [46, 15, 79]]


In [22]:
decoder_input = []
for line in lines['tar']:
    decoded_line = []
    for char in line:
        decoded_line.append(tar_to_index[char])
    decoder_input.append(decoded_line)

print(f'tar 문장의 정수 인코딩 {decoder_input[:5]}')

tar 문장의 정수 인코딩 [[68, 33, 85, 78, 33, 88, 33, 39], [68, 33, 1, 78, 53, 84, 67, 16, 100, 33, 39], [68, 33, 82, 42, 33, 53, 44, 23, 52, 16, 33, 88, 33, 39], [68, 33, 12, 44, 23, 58, 16, 33, 88, 33, 39], [68, 33, 11, 78, 6, 23, 52, 33, 88, 33, 39]]


In [23]:
decoder_target = []
for line in lines['tar']:
    char_position = 0
    # 문자 위치를 추적할 수 있는 변수
    decoded_line = []
    for char in line:
        if char_position != 0:
            decoded_line.append(tar_to_index[char])
        char_position += 1
    decoder_target.append(decoded_line)

print(f'디코더 target 문장의 정수 인코딩 {decoder_target[:5]}')

디코더 target 문장의 정수 인코딩 [[33, 85, 78, 33, 88, 33, 39], [33, 1, 78, 53, 84, 67, 16, 100, 33, 39], [33, 82, 42, 33, 53, 44, 23, 52, 16, 33, 88, 33, 39], [33, 12, 44, 23, 58, 16, 33, 88, 33, 39], [33, 11, 78, 6, 23, 52, 33, 88, 33, 39]]


In [24]:
lengths = []
for line in lines['src']:
    lengths.append(len(line))

max_src = max(lengths)
print(f'src 문장의 최대 길이 {max_src}')

src 문장의 최대 길이 22


In [25]:
lengths = []
for line in lines['tar']:
    lengths.append(len(line))

max_tar = max(lengths)
print(f'tar 문장의 최대 길이 {max_tar}')

tar 문장의 최대 길이 76


In [26]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar, padding='post')
# 인코더와 디코더에 들어갈 데이터 패딩 작업

# input과 output의 길이를 맞춰줄 필요가 없다 > 언어에 따른 길이의 차이가 있기 때문에

In [27]:
decoder_target = pad_sequences(decoder_target, maxlen=max_tar, padding='post')
# 디코더에 정답 데이터로 사용될 데이터 패딩 작업

# pre: 문장 생성 시에는 문장 끝에 어떤 단어가 나올지가 더 중요하기 때문에 앞쪽에 패딩을 넣어주는 것이 좋다
# post: 문장의 순서, 시퀀스를 유지하는 것이 더 중요한 예측에서는 뒷쪽에 패딩을 넣어주는 것이 좋다

# target maxlen=max_tar: 패딩은 부족한 부분을 0을 채워 동일한 길이로 맞추는 과정이기 때문에, target과 decoder의 길이 차이는 중요하지 않다

In [28]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [29]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [30]:
encoder_inputs = Input(shape=(None, src_vocab_size))
# (None, src_vocab_size)의 형태를 갖는 input을 주겠다
# None: 가변적인 입력 문장의 길이 (명시하지 않겠다)
# abc > [[0, 0, 1], [0, 1, 0], [1, 0, 0]], de > [[1, 0, 1]. [1, 1, 1]]

In [31]:
encoder_lstm = LSTM(units=256, return_state=True)

In [32]:
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# LSTM은 은닉 상태와 셀 상태를 리턴한다
# state_h: 은닉 상태 (현재 시점에서의 출력), state_c: 장기기억으로 보존하고 있는 셀 상태 (비교적 중요한 값들을 기억하고 있다)

In [33]:
encoder_states = [state_h, state_c]
# context vector

In [34]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
# 디코더는 인코더의 마지막 context vector를 초기 은닉 상태로 사용한다

In [35]:
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
# return_sequences: 모든 타임스텝의 출력
# return_state: 마지막 타임스텝의 은닉 상태와 셀 상태를 출력

In [36]:
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 outputs 출력되는 구간
# state_h, state_c를 사용하지 않으므로 _로 넣어둔다 (정답을 넣어줄 것이기 때문 - 교사강요)

In [37]:
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
# 디코더의 출력은 타겟 단어 집합의 각 단어에 대한 확률 중 높은 값이어야한다

In [38]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [39]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# 모델에 입력할 데이터의 형태
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [40]:
model.fit(
    x=[encoder_input, decoder_input],
    # 모델에 입력할 데이터
    y=decoder_target,
    batch_size=64,
    epochs=20,
    validation_split=0.2
)

Epoch 1/20
750/750 [==============================] - 196s 257ms/step - loss: 0.7337 - val_loss: 0.6377
Epoch 2/20
750/750 [==============================] - 201s 267ms/step - loss: 0.4467 - val_loss: 0.5110
Epoch 3/20
750/750 [==============================] - 199s 265ms/step - loss: 0.3724 - val_loss: 0.4519
Epoch 4/20
750/750 [==============================] - 205s 273ms/step - loss: 0.3302 - val_loss: 0.4149
Epoch 5/20
750/750 [==============================] - 214s 285ms/step - loss: 0.3021 - val_loss: 0.3919
Epoch 6/20
750/750 [==============================] - 220s 294ms/step - loss: 0.2815 - val_loss: 0.3741
Epoch 7/20
750/750 [==============================] - 229s 306ms/step - loss: 0.2653 - val_loss: 0.3609
Epoch 8/20
750/750 [==============================] - 226s 302ms/step - loss: 0.2521 - val_loss: 0.3527
Epoch 9/20
750/750 [==============================] - 225s 299ms/step - loss: 0.2413 - val_loss: 0.3458
Epoch 10/20
750/750 [==============================] - 215s 286m

In [41]:
# 번역 동작 step
# 1. 번역하려고 하는 input 문장이 인코더에 들어가서 context 벡터를 얻는다
# 2. context 벡터와 <sos> (\t)를 디코더로 보낸다
# 3. 디코더가 <eos> (\n)이 나올 때까지 다음 문자를 예측 반복

encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [42]:
decoder_state_input_h = Input(shape=(256, ))  # 은닉 상태
decoder_state_input_c = Input(shape=(256, ))  # 셀 상태

# 디코더 셀에서 각각 이전 시점의 상태를 저장하는 텐서
# 디코더의 은닉 상태와 셀 상태를 입력으로 받기 위한 텐서로, 디코더 LSTM의 hidden units의 크기와 동일하게 넣어주기

In [43]:
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [44]:
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 디코더 LSTM 모델이 입력 값인 decoder_inputs와, 이전 상태인 decoder_states_inputs를 입력 받아 다음 단어를 예측하는 출력(decoder_outputs)와 새로운 은닉상태/셀상태(state_h, state_c)를 계산

In [45]:
decoder_states = [state_h, state_c]

In [46]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
# 디코더 출력 값을 소프트맥스 레이어로 통과시켜 단어별 확률 분포 변화

In [47]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
# 다음 단어의 확률 분포와 새로운 상태 값을 반환

In [48]:
index_to_src = {}
for char, i in src_to_index.items():
    index_to_src[i] = char
    # key: 정수인덱스, value: 문자가 되도록 뒤집어주기

print(index_to_src)

{1: 'M', 2: 'q', 3: "'", 4: 'l', 5: 'P', 6: '0', 7: '7', 8: '"', 9: 'S', 10: 'B', 11: 'b', 12: 'X', 13: 'e', 14: 'I', 15: 'i', 16: '-', 17: 'é', 18: 'u', 19: '3', 20: 'N', 21: 'G', 22: 'C', 23: '5', 24: 'p', 25: '8', 26: 'F', 27: '€', 28: ' ', 29: '$', 30: 'z', 31: 'O', 32: 'w', 33: 's', 34: 'n', 35: 'A', 36: 'o', 37: 'R', 38: 'y', 39: '6', 40: ',', 41: 't', 42: 'r', 43: '%', 44: '4', 45: 'x', 46: 'H', 47: 'g', 48: 'k', 49: 'L', 50: 'f', 51: 'j', 52: '’', 53: 'W', 54: 'h', 55: 'Y', 56: 'Q', 57: '/', 58: 'ï', 59: '9', 60: 'v', 61: 'm', 62: 'T', 63: '&', 64: 'a', 65: 'D', 66: 'E', 67: 'c', 68: 'V', 69: 'U', 70: '!', 71: 'J', 72: 'd', 73: ':', 74: '1', 75: 'K', 76: 'Z', 77: '?', 78: '2', 79: '.'}


In [49]:
index_to_tar = {}
for char, i in tar_to_index.items():
    index_to_tar[i] = char

print(index_to_tar)

{1: 'M', 2: '«', 3: '‽', 4: 'q', 5: "'", 6: 'l', 7: 'P', 8: '0', 9: '7', 10: '"', 11: 'S', 12: 'B', 13: 'ê', 14: 'b', 15: 'X', 16: 'e', 17: 'I', 18: 'i', 19: 'î', 20: 'œ', 21: '-', 22: 'é', 23: 'u', 24: '3', 25: 'N', 26: 'G', 27: 'C', 28: '5', 29: 'p', 30: '8', 31: 'ë', 32: 'F', 33: ' ', 34: '$', 35: 'û', 36: 'z', 37: 'O', 38: 'w', 39: '\n', 40: 'Ô', 41: 's', 42: 'n', 43: 'A', 44: 'o', 45: 'R', 46: '\xa0', 47: 'y', 48: 'Ê', 49: '6', 50: 'ç', 51: ',', 52: 't', 53: 'r', 54: '%', 55: '4', 56: 'x', 57: 'H', 58: 'g', 59: '\u202f', 60: 'k', 61: 'L', 62: 'è', 63: 'j', 64: 'f', 65: '’', 66: 'W', 67: 'h', 68: '\t', 69: '\u2009', 70: 'ï', 71: 'Q', 72: 'Y', 73: '9', 74: 'v', 75: 'm', 76: 'T', 77: '&', 78: 'a', 79: 'D', 80: 'É', 81: 'Ç', 82: 'E', 83: 'ô', 84: 'c', 85: 'V', 86: 'U', 87: 'à', 88: '!', 89: 'J', 90: 'ù', 91: 'd', 92: 'À', 93: ':', 94: '1', 95: 'K', 96: '?', 97: '2', 98: 'â', 99: '‘', 100: '.', 101: '»'}


In [50]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, tar_vocab_size))
    # 크기가 (1, 1, tar_vocab_size)인 배열 생성
    target_seq[0, 0, tar_to_index['\t']] = 1
    # 0번째 문장의 0번째 단어의 \t에 해당하는 위치를 1로 설정

    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # target_sequence: 현재 디코더의 입력 시퀀스
        # states_value: context vector
        # output_tokens: 다음 단어에 대한 예측 확률 분포, h: 은닉 상태, c: 셀 상태

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # 예측된 데이터 중에서 현재 타임 스텝인 마지막 값을 불러와서 그 중에 argmax(최댓값)을 sampled_token_index에 저장
        sampled_char = index_to_tar[sampled_token_index]

        decoded_sentence += sampled_char

        if sampled_char == '\n' or len(decoded_sentence) > max_tar:
            stop_condition = True
            # eos에 도달하거나 최대 문장 길이를 넘어서면 반복을 중단

        target_seq = np.zeros([1, 1, tar_vocab_size])
        # 1, 1, target_vocab_size의 배열 생성
        # (배치 사이즈, 시퀀스 길이, 타겟 집합의 크기)
        target_seq[0, 0, sampled_token_index] = 1
        # 첫 번째 배치, 첫 번째 타임스텝에서 sampled_token_index에 해당하는 위치를 1로 설정

        states_value = [h, c]
        # 현재 시점의 상태를 다음 시점의 상태로 전달

    return decoded_sentence

In [53]:
for seq_index in [128, 30, 14, 554]:
    input_seq = encoder_input[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print(f'입력 문장: {lines.src[seq_index]}')
    print(f'정답 문장: {lines.tar[seq_index][2:len(lines.tar[seq_index])-1]}')
    print(f'번역 문장: {decoded_sentence[1:len(decoded_sentence)]}')

1/1 [==============================] - 0s 18ms/step
입력 문장: I'm OK.
정답 문장: Je vais bien. 
번역 문장: Je suis content. 

1/1 [==============================] - 0s 17ms/step
입력 문장: Help!
정답 문장: À l'aide ! 
번역 문장: Salut ! 

1/1 [==============================] - 0s 23ms/step
입력 문장: Run.
정답 문장: Cours ! 
번역 문장: Courrasse-le. 

1/1 [==============================] - 0s 17ms/step
입력 문장: Get down.
정답 문장: Descendez ! 
번역 문장: Descends ! 

